In [107]:
import pymongo
import pandas as pd
import matplotlib as plt
import json
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures2

market = futures.market2
unit = futures.unit
position = futures.position2

date = "202000120"
#查询会员
BrokerID='海通期货'
# BrokerID='永安期货'
#加载数据
market = DataFrame(list(market.find({'date': {'$gte': date}})))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find({'date': {'$gte': date}}))).dropna()

# #选择需要显示的字段
market = market.copy()
# 指数收盘
market['cv'] = market.apply(lambda x: x['close'] * x['open_interest'], axis=1)
closes = market.groupby(['date', 'variety'])[['cv', 'open_interest']].sum()
#待解决问题：根据最小价位变动，给收盘价做五舍四入处理
closes['close_index'] = closes['cv'] / closes['open_interest']
market = closes.reset_index()[['date','variety','close_index']]

unit=unit[[ 'variety','unit']]

# #汇总合约
data3=position[(position['long_party_name'] == BrokerID)]
data3=data3[['date','variety','long_party_name','long_open_interest']]
data3=data3.groupby(['date','variety','long_party_name'])[['long_open_interest']].sum()

data4=position[(position['short_party_name'] == BrokerID)]
data4=data4[['date','variety','short_party_name','short_open_interest']]
data4=data4.groupby(['date','variety','short_party_name'])[['short_open_interest']].sum()


# #并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: BrokerID,axis=1)
data5.tail(100)
#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['long_open_interest']-x['short_open_interest'],axis=1)

# df['当日涨幅']=df.apply(lambda x: x['收盘']-x['收盘'].shift(1)-1,axis=1)
#选择需要显示的字段
data5=data5[['会员简称','long_open_interest','short_open_interest','净持仓']]
data5=data5.reset_index(['variety','date'])
netpostion=data5.set_index('date')


netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)
# print(netpostion)
#合约价值
contractValue=pd.merge(market,unit,how='left',sort=False).drop_duplicates()

contractValue['contractValue'] = contractValue.apply(lambda x: x['close_index']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]

# #值替换replace()
# # contractValue=contractValue.replace(['TA'],'PTA')
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

# #净持仓价值
sz['净持仓价值']=sz.apply(lambda x: int(x['净持仓']*x['contractValue']/10000),axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()
sz['化工板块']=sz[['RU','MA','V','L','PP','BU','TA']].sum(axis=1)
sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
sz['黑色板块']=sz[['RB','HC','ZC','J','JM','I']].sum(axis=1)
sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','TA','MA','PP','JM','J','ZC','I','BU','HC','SM','SF','FU']].sum(axis=1)
sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
sz['商品板块']=sz.sum(axis=1)


# chemical=sz.copy()
# chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
# chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
# chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
# chemical['20200120':]
# chemical = chemical.reset_index()

# # futures = client.futures
# # flows = futures.chemical
# # flows.insert(json.loads(chemical.T.to_json()).values())
print("计算完毕")


计算完毕


In [108]:
chemical=sz.copy()
chemical=chemical[['RU','MA','V','L','PP','BU','TA','商品板块','工业品','化工板块']]
chemical['max']=chemical[['RU','MA','V','L','PP','BU','TA']].idxmax(axis=1)
chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
chemical['20200120':]

,variety,RU,MA,V,L,PP,BU,TA,商品板块,工业品,化工板块,max,min
date,会员简称,,,,,,,,,,,,
20200120,海通期货,-73827,49429,-8052,13056,-12471,20610,39034,1008142,9220,27779,MA,RU
20200121,海通期货,-38453,-18407,-4455,24644,-43773,18383,0,79942,-120182,-62061,L,PP
20200122,海通期货,-28803,-33581,-5444,22154,-38909,16145,0,206284,-62259,-68438,L,PP
20200123,海通期货,-42959,1474,-7702,21821,-32251,9612,0,286288,-39055,-50005,L,RU
20200203,海通期货,-39183,-9866,-16355,0,-29970,9073,0,29517,-56391,-86301,BU,RU
20200204,海通期货,-54062,-3462,-16919,0,-18543,0,34759,-3373,83745,-58227,TA,RU
20200205,海通期货,-85738,-19588,-17113,16741,-29627,10358,34295,-652475,-177738,-90672,TA,RU
20200206,海通期货,-87854,-11527,-19877,0,0,-1504,0,-773812,-239639,-120762,L,RU
20200207,海通期货,-82573,-18909,-17530,0,-25810,23853,0,-610976,-187515,-120969,BU,RU


In [85]:
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
黑色板块['20200120':]


,variety,RB,HC,ZC,J,JM,I,黑色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,,
20200120,永安期货,61480,13122,-6705,-72635,-81901,180395,93756,548184,3785629,I,JM
20200121,永安期货,45463,2563,-6203,-60355,-86991,156647,51124,331289,3159165,I,JM
20200122,永安期货,70796,5943,-7136,-12976,-81024,116554,92157,474251,4009399,I,JM
20200123,永安期货,132688,23694,-5963,-5536,-85961,88158,147080,482264,4075371,RB,JM
20200203,永安期货,100266,26720,-30650,44458,-60781,84799,164812,501813,4551624,RB,JM
20200204,永安期货,41680,23761,-15079,16160,-49908,-41911,-25297,439182,3927802,RB,JM
20200205,永安期货,34373,31593,-37891,35588,-40204,-35175,-11716,572394,4253573,J,JM
20200206,永安期货,-4777,30339,-36234,54276,-23809,-64894,-45099,680931,4902014,J,I
20200207,永安期货,-53010,31144,-61079,47235,-22509,-75900,-134119,539932,4189031,J,I


In [64]:
# import json
# from pymongo import MongoClient
# # 连接数据库
# client = MongoClient('localhost', 27017)
# db = client.futures
# flow = db.flow
# 黑色板块=sz.copy()
# 黑色板块=黑色板块[['RB','HC','ZC','J','JM','I','黑色板块','工业品','商品板块']] 
# 黑色板块['max']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmax(axis=1) 
# 黑色板块['min']=黑色板块[['RB','HC','ZC','J','JM','I']].idxmin(axis=1)
# df = 黑色板块.reset_index()
# flow.insert_many(json.loads(df.T.to_json()).values())
# print("写入OK")

In [86]:
oil=sz.copy()
oil=oil[['Y','P','OI','油脂板块','农产品']]
oil['max']=oil[['Y','P','OI']].idxmax(axis=1) 
oil['min']=oil[['Y','P','OI']].idxmin(axis=1)
oil['20200120':]

,variety,Y,P,OI,油脂板块,农产品,max,min
date,会员简称,,,,,,,
20200120,永安期货,395777,147491,12585,555853,868632,Y,OI
20200121,永安期货,409360,103004,13769,526133,875730,Y,OI
20200122,永安期货,421335,161131,14365,596831,1028617,Y,OI
20200123,永安期货,448641,128137,17098,593876,1002939,Y,OI
20200203,永安期货,354640,58192,13001,425833,979867,Y,OI
20200204,永安期货,282397,37160,4821,324378,892836,Y,OI
20200205,永安期货,278061,48414,7889,334364,821803,Y,OI
20200206,永安期货,278388,105875,2904,387167,956192,Y,OI
20200207,永安期货,239973,92334,37447,369754,928700,Y,OI


In [89]:
有色=sz.copy()
有色=有色[['CU','AL','ZN','NI','PB','有色板块','工业品','商品板块']]
有色['max']=有色[['CU','AL','ZN','NI','PB']].idxmax(axis=1) 
有色['min']=有色[['CU','AL','ZN','NI','PB']].idxmin(axis=1)
有色['20200120':]

,variety,CU,AL,ZN,NI,PB,有色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,
20200120,永安期货,408192,34245,-28991,-68520,154,345080,548184,3785629,CU,NI
20200121,永安期货,316717,14880,-51847,-75761,-165,203824,331289,3159165,CU,NI
20200122,永安期货,384698,10882,-69517,-51014,1386,276435,474251,4009399,CU,ZN
20200123,永安期货,365854,16916,-48180,-88079,-460,246051,482264,4075371,CU,NI
20200203,永安期货,255283,24019,-42289,-17458,-370,219185,501813,4551624,CU,ZN
20200204,永安期货,327963,17149,-34880,-667,-623,308942,439182,3927802,CU,ZN
20200205,永安期货,329198,6097,-28484,13511,-13735,306587,572394,4253573,CU,ZN
20200206,永安期货,374440,10605,-1813,-3035,-8415,371782,680931,4902014,CU,PB
20200207,永安期货,338509,13950,-19193,14218,-16900,330584,539932,4189031,CU,ZN


In [90]:
农产品=sz.copy()
农产品=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP','农产品']]
农产品['max']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmax(axis=1) 
农产品['min']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmin(axis=1)
农产品['20200120':]

,variety,A,C,M,RM,Y,P,OI,CF,SR,JD,CS,AP,农产品,max,min
date,会员简称,,,,,,,,,,,,,,,
20200120,永安期货,36466,166246,65968,18616,395777,147491,12585,137361,-27210,-23563,-26792,-34313,868632,Y,AP
20200121,永安期货,31402,180727,103362,14819,409360,103004,13769,165892,-69088,-25583,-22193,-29741,875730,Y,SR
20200122,永安期货,31663,206588,73509,21502,421335,161131,14365,232954,-69220,-24640,-17148,-23422,1028617,Y,SR
20200123,永安期货,33864,209988,13904,21309,448641,128137,17098,256696,-68317,-29508,-9556,-19317,1002939,Y,SR
20200203,永安期货,31666,231572,70920,33616,354640,58192,13001,275920,-34612,-28819,-8340,-17889,979867,Y,SR
20200204,永安期货,31218,186617,76298,17153,282397,37160,4821,279643,31311,-25721,-2844,-25217,892836,Y,JD
20200205,永安期货,32003,184147,74667,16369,278061,48414,7889,207937,28834,-26481,-4731,-25306,821803,Y,JD
20200206,永安期货,31951,181943,84887,1798,278388,105875,2904,267930,55719,-24971,-4248,-25984,956192,Y,AP
20200207,永安期货,30702,164739,91659,10440,239973,92334,37447,255690,60058,-24941,-4043,-25358,928700,CF,AP


In [91]:
板块=sz.copy()
板块=板块[['商品板块','工业品','化工板块','黑色板块','有色板块','农产品','油脂板块']]
板块['20200120':]

,variety,商品板块,工业品,化工板块,黑色板块,有色板块,农产品,油脂板块
date,会员简称,,,,,,,
20200120,永安期货,3785629,548184,8474,93756,345080,868632,555853
20200121,永安期货,3159165,331289,5909,51124,203824,875730,526133
20200122,永安期货,4009399,474251,39644,92157,276435,1028617,596831
20200123,永安期货,4075371,482264,22052,147080,246051,1002939,593876
20200203,永安期货,4551624,501813,58461,164812,219185,979867,425833
20200204,永安期货,3927802,439182,111924,-25297,308942,892836,324378
20200205,永安期货,4253573,572394,225765,-11716,306587,821803,334364
20200206,永安期货,4902014,680931,271083,-45099,371782,956192,387167
20200207,永安期货,4189031,539932,260665,-134119,330584,928700,369754
